# Sistem Klasifikasi Anjing dan Kucing menggunakan Metode LBP

Coded By : Fauzan Firdaus (1301164317) & Nur Azizah Agustina (1301164076)

Pertama-tama, lakukan import terlebih dahulu untuk beberapa library yang akan digunakan.

In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import matplotlib

from skimage.color import rgb2gray
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import glob
from skimage.feature import local_binary_pattern

Definisikan fungsi 'extractLBP' untuk melakukan proses ekstraksi fitur dengan metode LBP.

In [2]:
def extractLBP(img):
    
    lbp = local_binary_pattern(img, 24,8, method="uniform")
    (hist, _) = np.histogram(lbp.ravel(),bins=np.arange(0, 27),range=(0, 26))                   
    hist = hist.astype("float")                         
    hist /= (hist.sum() + (1e-7))
    return hist

Setelah itu, fungsi 'featureExtraction' dibawah ini berfungsi untuk melakukan proses ekstraksi fitur bentuk secara iteratif untuk semua gambar dalam array.

In [3]:
def featureExtraction(arr):
    arr_feature=[]
    for i in range(np.shape(arr)[0]):
        arr_feature.append(extractLBP(arr[i]))
    return arr_feature

Fungsi selanjutnya adalah 'preprocessing' yang berfungsi untuk melakukan tahap preprocessing (membuat gambar menjadi abu-abu).

In [4]:
def preprocessing(arr):
    arr_prep=[]
    for i in range(np.shape(arr)[0]):
        img=cv2.cvtColor(arr[i], cv2.COLOR_BGR2GRAY)
        arr_prep.append(img)
    return arr_prep

Fungsi yang terakhir adalah 'loadData' yang berfungsi untuk memuat semua gambar/dataset dan labelnya menjadi bentuk array yang berada dalam folder Datasets.

In [5]:
def loadData(arr,labelobject):
    label=[]
    strr = "Datasets/"+str(labelobject)+"/*.jpg"
    for file in glob.glob(strr):
        img=np.asarray(plt.imread(file))
        arr.append(img)
        label.append(labelobject)
    return arr,label

Lakukan proses load gambar dan labelnya

In [6]:
cat=[]
dog=[]

cat,label_cat = loadData(cat,1)
dog,label_dog = loadData(dog,2)

Gabungkan semua dataset menjadi 1 array

In [7]:
combineImage = np.concatenate((cat,dog))
combineLabel = np.concatenate((label_cat,label_dog))

Pecah dataset menjadi 2 bagian, yaitu data train dan data test. Hal ini dilakukan bertujuan dalam proses memvalidasi dataset.

In [8]:
X_train, X_test, y_train, y_test = train_test_split(combineImage, combineLabel, test_size=0.2, random_state=42)

Lakukan preprocessing terlebih dahulu

In [9]:
X_trainp=preprocessing(X_train)
X_testp=preprocessing(X_test)

Lalu, lakukan proses ekstraksi.

In [10]:
X_trainext=featureExtraction(X_trainp)
X_testext=featureExtraction(X_testp)

Tahap akhir adalah melakukan learning (fit) model / data train. Setelah melakukan learning, proses yang dilakukan adalah melakukan prediksi label, lalu muncullah akurasi sebagai ketepatan dari prediksi label tersebut dengan label yang sebenarnya.

In [11]:
clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X_trainext, y_train)
result = clf.predict(X_testext)
print("Akurasi LBP:", accuracy_score(y_test, result) * 100)

Akurasi LBP: 67.75
